In [1]:
#loading libraries
import numpy as np
import pandas as pd
import sklearn as sk
import tensorflow as tf
import keras as k

from matplotlib import pyplot as plt
import seaborn as sns

from os import path

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
from os import path
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler

In [3]:
##printing the versions of different modules
print("Numpy ",np.version.version)
print("Pandas ",pd.__version__)
print("Sklearn ",sk.__version__)
print("Keras ", k.__version__)
print("TF ",tf.__version__)
if tf.test.gpu_device_name():
    print('    GPU: {}'.format(tf.test.gpu_device_name()))
else:
    print('    CPU version')
print()

Numpy  1.21.6
Pandas  1.3.5
Sklearn  1.0.2
Keras  2.3.1
TF  1.15.5
    CPU version




User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   OMP_NUM_THREADS=8

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_HAND_THREAD=false
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_IN

In [4]:
np.random.seed(7)
tf.set_random_seed(7) # tf.random.set_seed(7) # v2.x

In [5]:
filename = "./pima_diabetes.csv"
col_names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
df = pd.read_csv(filename,names = col_names)
print(df.head(3))

   preg  plas  pres  skin  test  mass   pedi  age  class
0     6   148    72    35     0  33.6  0.627   50      1
1     1    85    66    29     0  26.6  0.351   31      0
2     8   183    64     0     0  23.3  0.672   32      1


In [6]:
print("***df.shape \n", df.shape)
print("***df.columns \n", df.columns)
print("***df.dtypes \n", df.dtypes)
print("***df.describe() \n", df.describe())
print("***df.isnull().sum() \n", df.isnull().sum())

***df.shape 
 (768, 9)
***df.columns 
 Index(['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class'], dtype='object')
***df.dtypes 
 preg       int64
plas       int64
pres       int64
skin       int64
test       int64
mass     float64
pedi     float64
age        int64
class      int64
dtype: object
***df.describe() 
              preg        plas        pres        skin        test        mass  \
count  768.000000  768.000000  768.000000  768.000000  768.000000  768.000000   
mean     3.845052  120.894531   69.105469   20.536458   79.799479   31.992578   
std      3.369578   31.972618   19.355807   15.952218  115.244002    7.884160   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      1.000000   99.000000   62.000000    0.000000    0.000000   27.300000   
50%      3.000000  117.000000   72.000000   23.000000   30.500000   32.000000   
75%      6.000000  140.250000   80.000000   32.000000  127.250000   36.600000   
max     17.00000

In [7]:
from sklearn.model_selection import train_test_split

x = df.drop(columns=['class'], axis=1)
y = df['class']
print("***df.head(2) \n",df.head(2))
print("***x.head(2) \n", x.head(2))
print("***y.head(2) \n",y.head(2))

x_train, x_test_val, y_train, y_test_val = train_test_split(x, y, test_size=0.40, random_state=7)
x_val, x_test, y_val, y_test = train_test_split(x_test_val, y_test_val, test_size=0.40, random_state=7)


***df.head(2) 
    preg  plas  pres  skin  test  mass   pedi  age  class
0     6   148    72    35     0  33.6  0.627   50      1
1     1    85    66    29     0  26.6  0.351   31      0
***x.head(2) 
    preg  plas  pres  skin  test  mass   pedi  age
0     6   148    72    35     0  33.6  0.627   50
1     1    85    66    29     0  26.6  0.351   31
***y.head(2) 
 0    1
1    0
Name: class, dtype: int64


In [8]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

In [9]:
from keras.models import Model
from keras.layers import Input, Dense

def model_2():
    input_layer = Input(shape=(8,))
    hidden_layer = Dense(8, activation='relu')(input_layer)
    output_layer = Dense(1, activation='sigmoid')(hidden_layer)

    model = Model(inputs=input_layer, outputs=output_layer)

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [10]:
model= model_2()
history = model.fit(x_train,y_train, epochs=150,batch_size=10, validation_data=(x_val, y_val)) #

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 460 samples, validate on 184 samples


2024-07-04 16:39:12.729511: I tensorflow/core/common_runtime/process_util.cc:136] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/150
310/460 [===================>..........] - ETA: 0s - loss: 0.7495 - accuracy: 0.4290 

OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-7
OMP: Info #156: KMP_AFFINITY: 8 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #179: KMP_AFFINITY: 1 packages x 4 cores/pkg x 2 threads/core (4 total cores)
OMP: Info #214: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #171: KMP_AFFINITY: OS proc 0 maps to package 0 core 0 thread 0 
OMP: Info #171: KMP_AFFINITY: OS proc 4 maps to package 0 core 0 thread 1 
OMP: Info #171: KMP_AFFINITY: OS proc 1 maps to package 0 core 1 thread 0 
OMP: Info #171: KMP_AFFINITY: OS proc 5 maps to package 0 core 1 thread 1 
OMP: Info #171: KMP_AFFINITY: OS proc 2 maps to package 0 core 2 thread 0 
OMP: Info #171: KMP_AFFINITY: OS proc 6 maps to package 0 core 2 thread 1 
OMP: Info #171: KMP_AFFINITY: OS proc 3 maps to package 0 core 3 thread 0 
OMP: Info #171: KMP_AFFINITY: O

460/460 [==============================] - 1s 2ms/step - loss: 0.7312 - accuracy: 0.4522 - val_loss: 0.7441 - val_accuracy: 0.4674
Epoch 2/150
460/460 [==============================] - 0s 183us/step - loss: 0.6832 - accuracy: 0.5500 - val_loss: 0.7063 - val_accuracy: 0.5543
Epoch 3/150
460/460 [==============================] - 0s 174us/step - loss: 0.6468 - accuracy: 0.6500 - val_loss: 0.6770 - val_accuracy: 0.6141
Epoch 4/150
460/460 [==============================] - 0s 182us/step - loss: 0.6186 - accuracy: 0.7065 - val_loss: 0.6510 - val_accuracy: 0.6196
Epoch 5/150
460/460 [==============================] - 0s 177us/step - loss: 0.5941 - accuracy: 0.7217 - val_loss: 0.6296 - val_accuracy: 0.6304
Epoch 6/150
460/460 [==============================] - 0s 171us/step - loss: 0.5730 - accuracy: 0.7283 - val_loss: 0.6109 - val_accuracy: 0.6467
Epoch 7/150
460/460 [==============================] - 0s 186us/step - loss: 0.5547 - accuracy: 0.7326 - val_loss: 0.5956 - val_accuracy: 0.6630

In [11]:
evaluator = KerasClassifier(build_fn =model_2 ,epochs = 150 , batch_size = 5)

In [12]:
random_seed = 7
kfold = KFold(n_splits = 10 , shuffle = True ,random_state = random_seed)

In [13]:
results = cross_val_score(evaluator,x_test , y_test ,cv = kfold)
print("model_2: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/150
111/111 [==============================] - 1s 5ms/step - loss: 0.7962 - accuracy: 0.5225
Epoch 2/150
111/111 [==============================] - 0s 447us/step - loss: 0.7658 - accuracy: 0.5315
Epoch 3/150
111/111 [==============================] - 0s 430us/step - loss: 0.7396 - accuracy: 0.5676
Epoch 4/150
111/111 [==============================] - 0s 447us/step - loss: 0.7174 - accuracy: 0.5676
Epoch 5/150
111/111 [==============================] - 0s 450us/step - loss: 0.6969 - accuracy: 0.5766
Epoch 6/150
111/111 [==============================] - 0s 512us/step - loss: 0.6795 - accuracy: 0.6306
Epoch 7/150
111/111 [==============================] - 0s 447us/step - loss: 0.6645 - accuracy: 0.6486
Epoch 8/150
111/111 [==============================] - 0s 425us/step - loss: 0.6488 - accuracy: 0.6667
Epoch 9/150
111/111 [==============================] - 0s 469us/step - loss: 0.6357 - accuracy: 0.6486
Epoch 10/150
111/111 [==============================] - 0s 437us/step - los

In [14]:
score = model.evaluate(x_test,y_test)
print(model.metrics_names[1], score[1]*100)

124/124 [==============================] - 0s 44us/step
accuracy 80.64516186714172
